In [8]:
# import libraries
import pandas as pd
import numpy as np
import json
from IPython.display import JSON
import nltk
import re
import string
from tqdm import tqdm
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from langdetect import detect

from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from scipy.sparse import save_npz, load_npz

# import own functions
import preprocessing

### Parse json files containing questions from consumers and experts

In [ ]:
filepath = 'data/EPIC-QA/consumer_questions_primary.json'
consumer_df = preprocessing.parse_questions(filepath)
consumer_df.head()

In [ ]:
# consumer_df.to_csv('data/processed/questions_consumer.csv', sep='\t')

In [ ]:
# parse json files containing questions from consumers and experts
filepath = 'data/EPIC-QA/expert_questions_primary.json'
expert_df = preprocessing.parse_questions(filepath)
expert_df.head()

In [ ]:
# expert_df.to_csv('data/processed/questions_expert.csv', sep='\t')

### Load and clean metadata file

In [2]:
metadata = preprocessing.clean_metadata('data/cord-19/metadata.csv')

CSV file loaded successfully
Dropping duplicate uids


/mnt/c/Users/omlea/Documents/_training/Lighthouse/lh-final-project/covid-lit-search/preprocessing.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates_df['nans'] = duplicates_df.apply(lambda x: x.isnull().sum(), axis=1)
  0%|          | 42/24167 [00:00<01:56, 206.23it/s]

Choosing rows to drop


100%|██████████| 24167/24167 [02:05<00:00, 191.86it/s]


Metadata cleaning complete


In [3]:
# metadata.to_csv('data/processed/metadata_clean.csv', sep='\t')

### Create cleaned title and abstract strings

In [2]:
metadata = preprocessing.load_cleaned_metadata('data/processed/metadata_clean.csv')

In [3]:
metadata.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,OBJECTIVE: This retrospective chart review des...,2001-07-04 00:00:00,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,none,none,none,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,none
1,02tnwd4m,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,Inflammatory diseases of the respiratory tract...,2000-08-15 00:00:00,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,none,none,none,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,none
2,ejv2xln0,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,Surfactant protein-D (SP-D) participates in th...,2000-08-25 00:00:00,"Crouch, Erika C",Respir Res,none,none,none,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,none
3,2b73a28n,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22 00:00:00,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,none,none,none,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,none
4,9785vg6d,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11 00:00:00,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,none,none,none,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,none


In [6]:
metadata['search_text'] = preprocessing.make_search_documents(metadata)

100%|██████████| 508070/508070 [27:58<00:00, 302.72it/s] 


In [12]:
def language(row):
    s = row.title + " " + row.abstract
    return detect(s)

In [13]:
row = metadata.iloc[0]
language(row)

'en'

In [15]:
metadata['language'] = [language(metadata.iloc[i]) for i in tqdm(range(len(metadata)))]

  1%|          | 4068/508070 [00:31<1:05:21, 128.54it/s]

KeyboardInterrupt: 

  1%|          | 4068/508070 [00:50<1:05:21, 128.54it/s]

In [7]:
# metadata.to_csv('data/processed/01_full_table.csv', sep='\t')

In [16]:
# # remove non-english (redundant)

# df = pd.read_csv('data/processed/01_full_table.csv', index_col=0)

# langs = pd.read_csv('data/processed/languages.csv', index_col=0)['0']
# df['language'] = langs
# df = df[df.language == 'en'].drop(columns='language')

# # remove remaining null rows
# df.dropna(inplace=True)